In [1]:
import pandas as pd
from ydata_profiling import ProfileReport
import jellyfish
import re

In [2]:
df_LOB = pd.read_csv('dataset/Legally_Operating_Businesses_20240413.csv', dtype=str)
df_LOB.columns

Index(['DCA License Number', 'License Type', 'License Expiration Date',
       'License Status', 'License Creation Date', 'Industry', 'Business Name',
       'Business Name 2', 'Address Building', 'Address Street Name',
       'Secondary Address Street Name', 'Address City', 'Address State',
       'Address ZIP', 'Contact Phone Number', 'Address Borough',
       'Borough Code', 'Community Board', 'Council District', 'BIN', 'BBL',
       'NTA', 'Census Tract', 'Detail', 'Longitude', 'Latitude', 'Location'],
      dtype='object')

In [ ]:
profile = ProfileReport(df_LOB, title="Pandas Profiling Report", explorative=True)
profile.to_notebook_iframe()

In [3]:
def null_detect(df, column):
    explicit_null_list = ['', 'N/A', '#N/A', 'n/a', 'No Data', 'Not Applicable']
    disguised_null_list = ['01/01/1900', '999-999-9999']
    for index, value in df[column].items():
        if pd.isnull(value) or value in explicit_null_list + disguised_null_list:
            df.loc[index, column] = 'Unknown'
    return df

for column in df_LOB.columns:
    df_LOB = null_detect(df_LOB, column)
df_LOB

,DCA License Number,License Type,License Expiration Date,License Status,License Creation Date,Industry,Business Name,Business Name 2,Address Building,Address Street Name,...,Community Board,Council District,BIN,BBL,NTA,Census Tract,Detail,Longitude,Latitude,Location
0,0967332-DCA,Business,02/28/2017,Inactive,02/04/2010,Home Improvement Contractor,"BARBARINO, JOHN JR.",JOHN BARBARINO JR_HOME IMPROVEMENT,239,MEDFORD CT,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
1,2103411-DCA,Individual,03/31/2024,Active,01/11/2022,Sightseeing Guide,"Nieciak, Joseph",Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
2,2060087-DCA,Individual,03/31/2024,Active,10/31/2017,Sightseeing Guide,"Arcaro, Tyler",Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
3,2078026-DCA,Individual,03/31/2024,Active,09/11/2018,Sightseeing Guide,"HESSE, DAPHNEE",Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
4,2057801-DCA,Individual,02/28/2019,Inactive,09/01/2017,Home Improvement Salesperson,"ESCOBAR, KEVIN",Unknown,Unknown,Unknown,...,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281408,2041725-DCA,Business,09/05/2016,Inactive,08/01/2016,Amusement Device Temporary,WONDERLAND MIDWAY CORP.,Unknown,23602,HILLSIDE AVE,...,411,19,4315239,4079550100,QN45,1483,"Device Name(s): CAROUSEL, ZAMPERLA KITE, FERR...",-73.7504924256429,40.77186106910604,"(40.77186106910604, -73.7504924256429)"
281409,1456246-DCA,Business,04/30/2024,Active,03/19/2013,Tow Truck Company,TYC GROUP INC.,DREAMWORK TOWING,961,39TH ST,...,312,38,3253329,3055820045,Unknown,Unknown,"Active Plate Numbers: 8755, 8754, 8756, 8753,...",-73.99357859645015,40.64503754359486,"(40.64503754359486, -73.99357859645015)"
281410,1450484-DCA,Business,04/30/2020,Inactive,07/26/2013,Tow Truck Company,727 AUTOMOTIVE INC.,Unknown,2953,EDSON AVE,...,212,12,2062760,2047970016,BX13,46202,"Active Plate Numbers: 6757, 8427, 6777",-73.83440359876691,40.87086063691892,"(40.87086063691892, -73.83440359876691)"
281411,2016100-1-DCA,Business,12/31/2021,Inactive,12/02/2014,Tobacco Retail Dealer,82E DELI GROCERY INC,Unknown,82,E 18TH ST,...,314,40,3117080,3050970089,BK42,512,Unknown,-73.96302960844992,40.649146211582234,"(40.649146211582234, -73.96302960844992)"


In [4]:
df_CDR = pd.read_csv('dataset/Construction_Demolition_Registrants_20240413.csv', dtype=str)
df_CDR.columns

Index(['CREATED', 'BIC NUMBER', 'ACCOUNT NAME', 'TRADE NAME', 'ADDRESS',
       'CITY', 'STATE', 'POSTCODE', 'PHONE', 'EMAIL', 'APPLICATION TYPE',
       'DISPOSITION DATE', 'EFFECTIVE DATE', 'EXPIRATION DATE', 'RENEWAL',
       'EXPORT DATE', 'LATITUDE', 'LONGITUDE', 'COMMUNITY BOARD',
       'COUNCIL DISTRICT', 'CENSUS TRACT', 'BIN', 'BBL', 'NTA', 'BORO'],
      dtype='object')

In [ ]:
profile = ProfileReport(df_CDR, title="Pandas Profiling Report", explorative=True)
profile.to_notebook_iframe()

In [5]:
def misspelling_detect(df, parition_columns, target_column, window_size=3, threshold=5):

    blocked_records = {}
    for i in range(len(df)):
        key = tuple(df[column][i] for column in parition_columns)
        if key not in blocked_records:
            blocked_records[key] = []
        blocked_records[key].append(i)
    
    for key in blocked_records:
        blocked_records[key].sort()
    
    misspellings = []
    df[target_column].fillna('', inplace=True)
    for block in blocked_records.values():
        for i in range(0, len(block) - window_size, window_size):
            flag = False
            temp = []
            temp.append(block[i])
            for j in range(i + 1, i + 1 + window_size):
                temp.append(block[j])
                if (
                    flag == False and
                    jellyfish.levenshtein_distance(df[target_column][block[i]], df[target_column][block[j]]) in range(1, threshold + 1)
                ):
                    flag = True
            if flag == True:
                misspellings.append(temp)

    return misspellings

parition_columns = ['LATITUDE', 'LONGITUDE']
target_column = 'ADDRESS'
misspellings = misspelling_detect(df_CDR, parition_columns, target_column)
for misspelling in misspellings:
    print(df_CDR.loc[misspelling][target_column])

/var/folders/hn/k3tw5vhs7h74ccfln640dvbm0000gn/T/ipykernel_28189/2708338559.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[target_column].fillna('', inplace=True)


396       P.O. BOX 195
403    134 MILBAR BLVD
406      P.O. BOX 5010
409    601 COMMERCE RD
Name: ADDRESS, dtype: object
551                P.O. BOX 453
554               P.O. BOX 3013
555    537 GLENMORE AVENUE 1FHH
556      52-04 69TH STREET FL 2
Name: ADDRESS, dtype: object
846          P.O. BOX 756
850      124 COLIGNI AVE.
851    64 CROSS POND ROAD
853          P.O. BOX 393
Name: ADDRESS, dtype: object
1034              PO BOX 273
1035    772 DEER PARK AVENUE
1037            P.O. BOX 541
1038             27 KOOL PL.
Name: ADDRESS, dtype: object
1098              P.O. BOX 4238
1099    57 HUDSON AVENUE SUIT 3
1100            P.O. BOX 564045
1103        1460 MANATUCK BLVD.
Name: ADDRESS, dtype: object
1283          PO BOX 633
1284    567 FIFTH AVENUE
1286        P.O. BOX 970
1287       47 Spruce Ave
Name: ADDRESS, dtype: object
1694                P.O. BOX 453
1697               P.O. BOX 3013
1698    537 GLENMORE AVENUE 1FHH
1699      52-04 69TH STREET FL 2
Name: ADDRESS, dtype: obje

In [6]:
df_SSR = pd.read_csv('dataset/2010_-_2016_School_Safety_Report_20240413.csv', dtype=str)
df_SSR.columns

Index(['School Year', 'Building Code', 'DBN', 'Location Name', 'Location Code',
       'Address', 'Borough', 'Geographical District Code', 'Register',
       'Building Name', '# Schools', 'Schools in Building', 'Major N', 'Oth N',
       'NoCrim N', 'Prop N', 'Vio N', 'ENGroupA', 'RangeA', 'AvgOfMajor N',
       'AvgOfOth N', 'AvgOfNoCrim N', 'AvgOfProp N', 'AvgOfVio N',
       'Borough Name', 'Postcode', 'Latitude', 'Longitude', 'Community Board',
       'Council District ', 'Census Tract', 'BIN', 'BBL', 'NTA'],
      dtype='object')

In [ ]:
profile = ProfileReport(df_SSR, title="Pandas Profiling Report", explorative=True)
profile.to_notebook_iframe()

In [7]:
def vocabulary_misspelling_detect(df, column, vocab):
    violations = []
    for index, value in df[column].items():
        if pd.isnull(value):
            continue
        if value not in vocab:
            for v in vocab:
                if jellyfish.jaro_winkler_similarity(value, v) < 0.9:
                    break
            else:
                violations.append(index)
    return violations

with open('reference/NTAName.txt', 'r') as file:
    vocab_NTA = file.read().splitlines()
violations = vocabulary_misspelling_detect(df_SSR, 'NTA', vocab_NTA)
df_SSR.loc[violations]

,School Year,Building Code,DBN,Location Name,Location Code,Address,Borough,Geographical District Code,Register,Building Name,...,Borough Name,Postcode,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA


In [8]:
df_SSR = pd.read_csv('dataset/2010_-_2016_School_Safety_Report_20240413.csv', dtype=str)
df_SSR.columns

Index(['School Year', 'Building Code', 'DBN', 'Location Name', 'Location Code',
       'Address', 'Borough', 'Geographical District Code', 'Register',
       'Building Name', '# Schools', 'Schools in Building', 'Major N', 'Oth N',
       'NoCrim N', 'Prop N', 'Vio N', 'ENGroupA', 'RangeA', 'AvgOfMajor N',
       'AvgOfOth N', 'AvgOfNoCrim N', 'AvgOfProp N', 'AvgOfVio N',
       'Borough Name', 'Postcode', 'Latitude', 'Longitude', 'Community Board',
       'Council District ', 'Census Tract', 'BIN', 'BBL', 'NTA'],
      dtype='object')

In [9]:
def outlier_detect(df, column):
    violations = []
    df_col = pd.to_numeric(df[column], errors='coerce')
    df_zscore = (df_col - df_col.mean()) / df_col.std()
    threshold = 2.2
    violations = df[df_zscore.abs() > threshold].index.tolist()
    return violations

violations = outlier_detect(df_SSR, 'AvgOfVio N')
df_SSR.loc[violations]['AvgOfVio N']

535     3.56
538     5.13
553     3.56
570     3.56
574     5.13
        ... 
6232    3.94
6233    3.07
6245    3.94
6248    3.94
6257    5.29
Name: AvgOfVio N, Length: 142, dtype: object

In [10]:
df_CDR = pd.read_csv('dataset/Construction_Demolition_Registrants_20240413.csv', dtype=str)
df_CDR.columns

Index(['CREATED', 'BIC NUMBER', 'ACCOUNT NAME', 'TRADE NAME', 'ADDRESS',
       'CITY', 'STATE', 'POSTCODE', 'PHONE', 'EMAIL', 'APPLICATION TYPE',
       'DISPOSITION DATE', 'EFFECTIVE DATE', 'EXPIRATION DATE', 'RENEWAL',
       'EXPORT DATE', 'LATITUDE', 'LONGITUDE', 'COMMUNITY BOARD',
       'COUNCIL DISTRICT', 'CENSUS TRACT', 'BIN', 'BBL', 'NTA', 'BORO'],
      dtype='object')

In [11]:
def pattern_violation_detect(df, column, regexp):
    violations = []
    for index, value in df[column].items():
        if pd.isnull(value):
            continue
        if not re.match(regexp, value):
            violations.append(index)
    return violations

regexp_email = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
regexp_phone = r'^\(\d{3}\) \d{3}-\d{4}$'
regexp_zip = r'^\d{5}(-\d{4})?$'
regexp_date = r'^\d{2}/\d{2}/\d{4}$'

violations = pattern_violation_detect(df_CDR, 'POSTCODE', regexp_zip)
df_CDR.loc[violations]

,CREATED,BIC NUMBER,ACCOUNT NAME,TRADE NAME,ADDRESS,CITY,STATE,POSTCODE,PHONE,EMAIL,...,EXPORT DATE,LATITUDE,LONGITUDE,COMMUNITY BOARD,COUNCIL DISTRICT,CENSUS TRACT,BIN,BBL,NTA,BORO
514325,01/04/2019,BIC-497872,BAMM DUMPSTERS CORP.,NaN,155 CECILIA PLACE,OCEANSIDE,NY,11572``,(516) 608-1451,bammdump2@aol.com,...,01/04/2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
545659,04/01/2019,#B1',NEW YORK,NY,10033-,(212) 740-4866,jg,CL2 - Exempt,NaN,2018-07-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
577605,03/25/2019,#B1',NEW YORK,NY,10033-,(212) 740-4866,jg,CL2 - Exempt,NaN,2018-07-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
578570,03/26/2019,#B1',NEW YORK,NY,10033-,(212) 740-4866,jg,CL2 - Exempt,NaN,2018-07-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
579693,03/27/2019,#B1',NEW YORK,NY,10033-,(212) 740-4866,jg,CL2 - Exempt,NaN,2018-07-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956673,04/08/2024,BOX #197',Staten Island,NY,10312,(718) 705-8408,worldwideenvironmental@gmail.com,CL2 - Exempt,2023-06-08,2023-04-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1958640,04/09/2024,BOX #197',Staten Island,NY,10312,(718) 705-8408,worldwideenvironmental@gmail.com,CL2 - Exempt,2023-06-08,2023-04-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1959700,04/10/2024,BOX #197',Staten Island,NY,10312,(718) 705-8408,worldwideenvironmental@gmail.com,CL2 - Exempt,2023-06-08,2023-04-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1959986,04/11/2024,BOX #197',Staten Island,NY,10312,(718) 705-8408,worldwideenvironmental@gmail.com,CL2 - Exempt,2023-06-08,2023-04-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_SSR = pd.read_csv('dataset/2010_-_2016_School_Safety_Report_20240413.csv', dtype=str)
df_SSR.columns

Index(['School Year', 'Building Code', 'DBN', 'Location Name', 'Location Code',
       'Address', 'Borough', 'Geographical District Code', 'Register',
       'Building Name', '# Schools', 'Schools in Building', 'Major N', 'Oth N',
       'NoCrim N', 'Prop N', 'Vio N', 'ENGroupA', 'RangeA', 'AvgOfMajor N',
       'AvgOfOth N', 'AvgOfNoCrim N', 'AvgOfProp N', 'AvgOfVio N',
       'Borough Name', 'Postcode', 'Latitude', 'Longitude', 'Community Board',
       'Council District ', 'Census Tract', 'BIN', 'BBL', 'NTA'],
      dtype='object')

In [13]:
def constriant_violation_detection(df, left, right):
    violations = []
    if (df.groupby(left)[right].value_counts() == 1).all():
        return violations
    df_grouped = df.groupby(left)[right]
    for left_val, right_vals in df_grouped:
        if (right_vals.value_counts() != len(right_vals)).any():
            violations.append((left_val, right_vals))
    return violations

left = ['NTA']
right = ['Borough Name']
violations = constriant_violation_detection(df_SSR, left, right)
violations

[(('Marble Hill-Inwood                                                         ',),
       Borough Name
  720     MANHATTAN
  783     MANHATTAN
  784     MANHATTAN
  785     MANHATTAN
  839     MANHATTAN
  ...           ...
  6253    BRONX    
  6254    BRONX    
  6255    BRONX    
  6256    BRONX    
  6257    BRONX    
  
  [63 rows x 1 columns])]